# Multi Label Classification

When a car is dirty, it can be classified as having any combination of foreign object, dirt and pet hair. The following notebook aims to write the code to classify images with multi labels in this way.

Sources:
- https://boascents2.medium.com/step-by-step-multi-label-image-classification-with-pytorch-gpu-e34d0aa6d578
- ChatGPT!

## Structuring data correctly

The first step is ensuring the training data is structured so that multiple labels can be inputted.

In [ ]:
classes = [ 'foreign-object', 'dirt', 'hair']

## The model

In [5]:
# Attempting to actually train a model

import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision
import pathlib
from cnn_multi_label import ConvNet_Multi_Label

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(), # flips image with p=0.5 to augment data
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])
train_path = 'C:/Users/hanna/Desktop/git/interiorcardamage/Data2/raw/train'
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(root=train_path, transform=train_transformer),
    batch_size=32, shuffle=True, num_workers=4,
    pin_memory=True,
)
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
     
model=ConvNet_Multi_Label(num_classes=len(classes))
model.to(device)
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.BCEWithLogitsLoss()
num_epochs=10
for epoch in range(num_epochs):
    #model.train()
    #print("Starting epoch " + str(epoch))
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        # One-hot encode the target labels
        labels_one_hot = torch.zeros(labels.size(0), len(classes)).to(device)
        labels_one_hot.scatter_(1, labels.unsqueeze(1), 1)  # Perform one-hot encoding
    
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=loss_function(outputs,labels_one_hot)
        loss.backward()
        optimizer.step()
    print('Epoch '+str(epoch) + ' finished.')
    torch.save(model.state_dict(),'best_checkpoint_Data3.model')
    print("Model saved")



Starting epoch 0
Epoch 0 finished.
Model saved
Starting epoch 1
Epoch 1 finished.
Model saved
Starting epoch 2
Epoch 2 finished.
Model saved
Starting epoch 3
Epoch 3 finished.
Model saved
Starting epoch 4
Epoch 4 finished.
Model saved
Starting epoch 5
Epoch 5 finished.
Model saved
Starting epoch 6
Epoch 6 finished.
Model saved
Starting epoch 7
Epoch 7 finished.
Model saved
Starting epoch 8
Epoch 8 finished.
Model saved
Starting epoch 9
Epoch 9 finished.
Model saved


In [7]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np

# Define transformations for the test images (similar to training)
transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(), # flips image with p=0.5 to augment data
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

# Load the test data using ImageFolder
path = 'C:/Users/hanna/Desktop/git/interiorcardamage/Data2/raw/test'
test_dataset = ImageFolder(root=path, transform=transform)

# Create a DataLoader for the test data
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Create an instance of the ConvNet model
model = ConvNet_Multi_Label(num_classes=3)

# Load the saved trained model
model.load_state_dict(torch.load("best_checkpoint_Data3.model"))

# Set the device for testing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Put the model in evaluation mode
model.eval()

# Perform inference on the test data
predictions = []
with torch.no_grad():
    for images, _ in test_loader:
        # Move images to the device
        images = images.to(device)

        # Forward pass
        outputs = model(images)
        
        # Apply a threshold to convert probabilities to binary predictions
        threshold = 0.5
        predicted_labels = (outputs >= threshold).float()
        
        # Append the predicted labels to the list
        predictions.append(predicted_labels.cpu().numpy())

# Convert the list of predictions to a single numpy array
predictions = np.concatenate(predictions)

# Print the predicted labels for the test data
print(predictions)


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
